In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点 AI 模型监控设置表格模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_setup.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_setup.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何为表格模型设置Vertex AI模型监控。

了解更多关于[Vertex AI模型监控](https://cloud.google.com/vertex-ai/docs/model-monitoring)。

### 目标

在这个笔记本中，您将学习如何设置`Vertex AI模型监控`服务，以检测输入预测请求中的特征偏差和漂移。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI模型监控`
- `Vertex AI预测`
- `Vertex AI模型`资源
- `Vertex AI端点`资源

执行的步骤包括：

- 下载预先训练的自定义表格模型。
- 将预训练模型上传为`模型`资源。
- 将`模型`资源部署到`端点`资源。
- 配置`端点`资源进行模型监控。
    - 特征输入的偏差和漂移检测。
    - 特征归因的偏差和漂移检测。
- 通过发送1000个预测请求自动生成`输入模式`。
- 列出、暂停、恢复和删除监控作业。
- 用预定义的`输入模式`重新启动监控作业。
- 查看记录的监控数据。

### 模型

本教程使用一个预训练模型，模型的工件存储在一个公共的云存储桶中。

该模型基于[这篇博文](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。这个模型的想法是，您的公司拥有详细的日志数据，描述了游戏用户如何与网站互动。原始数据包含以下信息类别：

- 身份 - 独特的玩家身份号码
- 人口统计特征 - 关于玩家的信息，比如玩家所在地理区域
- 行为特征 - 玩家触发某些游戏事件的次数，比如达到新的水平的次数
- 流失倾向 - 这是标签或目标特征，它提供了该玩家可能流失（即停止成为活跃玩家）的估计概率。

### 费用 

此教程使用 Google Cloud 的计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing), 并使用[Pricing Calculator 价格计算器](https://cloud.google.com/products/calculator/)来根据您的预期使用量生成一个费用估算。

## 安装

安装执行此笔记本所需的软件包。

In [ ]:
# Install required packages.
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                 google-cloud-bigquery \
                                 tensorflow==2.7 \
                                 protobuf==3.20.3

仅限协作：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在开始之前

设置你的项目ID

**如果你不知道你的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的信息。

In [ ]:
REGION = "us-central1"

#### 用户电子邮件

设置您的用户电子邮件地址以接收监控警报。

In [ ]:
import os

USER_EMAIL = "[your-email-address]"  # @param {type:"string"}

if os.getenv("IS_TESTING"):
    USER_EMAIL = "noreply@google.com"

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经通过验证。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何将Cloud Storage权限授予您的服务帐号，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

### 关于服务账号和权限的注意事项

**默认情况下不需要配置**，如果遇到任何权限相关问题，请确保以上服务账号具有所需的角色：

|服务账号电子邮件|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine默认服务账号|Dataflow管理员、Dataflow工作者、存储管理员、BigQuery管理员、Vertex AI用户|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI 平台服务代理|Vertex AI 服务代理|

1. 进入 https://console.cloud.google.com/iam-admin/iam。
2. 勾选 "包含由谷歌提供的角色授予" 复选框。
3. 授予相应的角色。

### 使用来自不同项目的数据源
- 对于BigQuery数据源，请授予两个服务账号的 "BigQuery 数据查看器" 角色。
- 对于CSV数据源，请授予两个服务账号的 "存储对象查看器" 角色。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才能运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring
from google.cloud.aiplatform.explain.metadata.tf.v2 import \
    saved_model_metadata_builder

### 初始化 Vertex AI SDK 用于 Python

为您的项目和相应的存储桶初始化 Vertex AI SDK 用于 Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

### 创建 BigQuery 客户端

在本教程中，您将使用与之前用于训练预训练模型的相同公共 BigQuery 表中的数据。您将创建一个客户端接口，然后用它来访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

设置硬件加速器

您可以为预测设置硬件加速器（例如，GPUs），或选择不使用任何（CPU）。硬件加速器可以降低预测请求的延迟响应时间。在选择硬件加速器时，请考虑延迟与额外成本的权衡。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器映像，以及分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个支持 GPU 的容器映像，并将 4 个 Nvidia Tesla K80 GPU 分配给每个 VM，您可以指定：

(aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

查看[可用加速器的位置](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

否则，请指定`(None, None)`来使用一个在CPU上运行的容器映像。

In [ ]:
GPU = False
if GPU:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建 Docker 容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if GPU:
    DEPLOY_VERSION = "tf2-gpu.2-5"
else:
    DEPLOY_VERSION = "tf2-cpu.2-5"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `DEPLOY_COMPUTE` 以配置用于预测的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 内存
     - `n1-highmem`: 每个 vCPU 6.5GB 内存
     - `n1-highcpu`: 每个 vCPU 0.9GB 内存
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96\] 个

*注意：您也可以在训练和部署中使用 n2 和 e2 机器类型，但它们不支持 GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## 介绍Vertex AI模型监控

Vertex AI模型监控支持AutoML表格模型和自定义表格模型。您可以监控入站预测请求中特征的偏斜和漂移检测，或监控出站预测响应中特征归因（可解释AI）的偏斜和漂移检测--即归因对输出（预测）的贡献分布。

以下是启用模型监控的基本步骤：

1. 将`Vertex AI` AutoML或自定义表格模型部署到`Vertex AI端点`。
2. 配置模型监控规范。
3. 将模型监控规范上传到`Vertex AI端点`。
4. 上传或自动生成用于解析的`输入模式`。
5. 对于特征偏斜检测，上传训练数据以自动生成特征分布。
6. 对于特征归因，上传相应的`Vertex AI可解释性`规范。

一旦配置完成，您可以启用/禁用监控，更改警报并更新模型监控配置。

当启用模型监控时，采样的传入预测请求将被记录到一个BigQuery表中。然后对记录的请求中包含的输入特征值进行分析，以便定期查看偏斜或漂移。您可以设置采样率以监控模型的一部分生产输入，并设置监控间隔。

模型监控服务需要知道如何解析特征值，这称为输入模式。对于AutoML表格模型，输入模式会自动生成。对于自定义表格模型，服务会尝试从前1000个预测请求中自动推导输入模式。另外，您也可以上传输入模式。

对于偏斜检测，监控服务需要训练数据中值的统计分布基线。对于AutoML表格模型，这会自动推导。对于自定义表格模型，您需要将训练数据上传到服务中，并让服务自动推导分布。

对于特征归因偏斜和漂移检测，则需要为您部署的模型启用`Vertex AI可解释性`。

了解更多关于[介绍Vertex AI模型监控](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

为`Vertex Explainable AI`生成可解释的元数据

如果您想对输出预测（响应）的特征归因进行倾斜和漂移检测，您需要执行以下额外步骤：

- 为模型指定可解释性规范。
- 随后将模型作为`Vertex AI Model`资源上传时，包含可解释性规范。
- 随后将模型监控配置规范上传到相应的`Vertex AI Endpoint`资源时，包含可解释性目标配置。

作为第一步，您可以使用辅助方法`SavedModelMetadataBuilder()`为您的模型创建可解释的AI规范。

了解更多关于[可解释性人工智能介绍](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)。

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"

params = {"sampled_shapley_attribution": {"path_count": 10}}
explanation_parameters = aiplatform.explain.ExplanationParameters(params)

builder = saved_model_metadata_builder.SavedModelMetadataBuilder(
    model_path=MODEL_ARTIFACT_URI, outputs_to_explain=["churned_probs"]
)
explanation_metadata = builder.get_metadata_protobuf()

### 将模型工件上传为 `Vertex AI Model` 资源

接下来，您可以使用 `upload()` 方法将预训练的自定义表格模型工件上传为 `Vertex AI Model` 资源，并使用以下参数：

- `display_name`：`Model` 资源的人类可读名称。
- `artifact_uri`：模型工件的 Cloud Storage 位置。
- `serving_container_image`：在将模型部署到 `Vertex AI Endpoint` 资源时要使用的服务容器图像。
- `explanation_parameters`：配置模型预测解释的参数。
- `explanation_metadata`：描述模型输入和输出以进行解释的元数据。
- `sync`：是否等待过程完成，还是立即返回（异步）。

了解更多关于[将模型导入 Vertex AI](https://cloud.google.com/vertex-ai/docs/model-registry/import-model)。

In [ ]:
model = aiplatform.Model.upload(
    display_name="churn",
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=explanation_parameters,
    explanation_metadata=explanation_metadata,
    sync=True,
)

print(model)

### 部署 `Vertex AI Model` 资源到 `Vertex AI Endpoint` 资源

接下来，您可以使用 `deploy()` 方法将您的 `Vertex AI Model` 资源部署到 `Vertex AI Endpoint` 资源，使用以下参数：

- `deploy_model_display`: 部署模型的人类可读名称。
- `machine_type`: 每个 VM 节点实例的机器类型。
- `min_replica_count`: 为自动调整而要预留的最小节点数目。
- `max_replica_count`: 为自动调整而要预留的最大节点数目。
- `accelerator_type`: 每个提供的节点的 GPU 加速器类型，如有的话。
- `accelerator_count`: 每个提供的节点的 GPU 加速器数目，如有的话。

了解更多关于[使用 Vertex AI 部署模型](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

if GPU:
    endpoint = model.deploy(
        deployed_model_display_name="churn",
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name="churn",
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

## 配置监控任务

配置监控任务包括以下规范：

- `alert_config`：发送监控警报的电子邮件地址。
- `schedule_config`：分析预测的时间窗口。
- `logging_sampling_strategy`：抽样预测请求的速率。
- `drift_config`：监视的特征和漂移阈值。
- `skew_config`：监视的特征和偏斜阈值。

了解更多关于[监控特征偏斜和漂移](https://cloud.google.com/vertex-ai/docs/model-monitoring/using-model-monitoring)。

### 配置警报规范

首先，您可以使用以下设置配置`alerting_config`规范:

- `user_emails`: 发送警报的一个或多个电子邮件列表。
- `enable_logging`: 将检测到的异常流式传输到Cloud Logging。默认值为False。

了解更多关于[为模型监控作业配置警报](https://cloud.google.com/vertex-ai/docs/model-monitoring/using-model-monitoring#monitor-job)。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监控间隔规范

接下来，您将使用以下设置配置`schedule_config`规范：

- `monitor_interval`:设置模型监控作业的调度间隔时间，单位为小时。最小时间间隔为1小时。

*注意:* REST API中规定的单位是秒。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置抽样规范

接下来，您将使用以下设置配置`logging_sampling_strategy`规范：

- `sample_rate`: 按百分比（在0和1之间）随机抽样进行监控。 选择的样本将被记录到BigQuery表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### 配置漂移检测规范

接下来，您可以使用以下设置配置`drift_config`规范：

- `drift_thresholds`：一个键/值对的字典，其中键是要监视的特征输入以检测特征输入漂移。值是检测阈值。当未指定时，特征的默认漂移阈值为0.3（30%）。
- `attribute_drift_threshold`：一个键/值对的字典，其中键是要监视的特征以检测特征归因漂移。值是检测阈值。当未指定时，特征的默认漂移阈值为0.3（30%）。

*注意：*启用特征输入或特征归因的漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05
ATTRIBUTION_DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {
    "country": DRIFT_THRESHOLD_VALUE,
    "cnt_user_engagement": DRIFT_THRESHOLD_VALUE,
}

ATTRIBUTION_DRIFT_THRESHOLDS = {
    "country": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
    "cnt_user_engagement": ATTRIBUTION_DRIFT_THRESHOLD_VALUE,
}

drift_config = model_monitoring.DriftDetectionConfig(
    drift_thresholds=DRIFT_THRESHOLDS,
    attribute_drift_thresholds=ATTRIBUTION_DRIFT_THRESHOLDS,
)

### 配置偏斜检测规范

接下来，您可以使用以下设置配置`skew_config`规范：

- `data_source`：原始训练数据集的数据源。源的格式默认为BigQuery表。 否则，设置`data_format`必须设置为以下值之一。 数据的位置必须是Cloud Storage位置。
  - `csv`： 
  - `jsonl`：
  - `tf-record`：
- `skew_thresholds`：一个键/值对字典，其中键是要监视的特征输入的输入特征。 值是检测阈值。 如果未指定，则特征的默认偏斜阈值为0.3（30％）。
- `attribute_skew_thresholds`：一个键/值对字典，其中键是要监视特征属性偏斜的输入特征。 值是检测阈值。 如果未指定，则特征的默认偏斜阈值为0.3（30％）。
- `target_field`：训练数据集的目标标签

*注意：*启用偏斜检测是可选的。

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://mco-mm.bqmlga4.train"  # @param {type:"string"}
# Prediction target column name in training dataset.
TARGET = "churned"

SKEW_THRESHOLD_VALUE = 0.5

SKEW_THRESHOLDS = {
    "country": SKEW_THRESHOLD_VALUE,
    "cnt_user_engagement": SKEW_THRESHOLD_VALUE,
}

ATTRIBUTE_SKEW_THRESHOLDS = {
    "country": SKEW_THRESHOLD_VALUE,
    "cnt_user_engagement": SKEW_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI,
    skew_thresholds=SKEW_THRESHOLDS,
    attribute_skew_thresholds=ATTRIBUTE_SKEW_THRESHOLDS,
    target_field=TARGET,
)

### 组装目标规范

最后，您可以使用以下设置来组装目标规范 `objective_config`：

- `skew_detection_config`:（可选）用于倾斜检测配置的规范。
- `drift_detection_config`:（可选）用于漂移检测配置的规范。
- `explanation_config`:（可选）用于启用特征归因监控时解释的规范。

In [ ]:
explanation_config = model_monitoring.ExplanationConfig()

objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
    explanation_config=explanation_config,
)

### 创建监控作业

您可以使用`aiplatform.ModelDeploymentMonitoringJob.create()`方法创建一个监控作业，根据您的监控规范，参数如下：

- `display_name`：监控作业的易读名称。
- `project`：项目 ID。
- `region`：地区。
- `endpoint`：要启用监控的完全限定资源名称的`Vertex AI Endpoint`。
- `logging_sampling_strategy`：采样配置的规范。
- `schedule_config`：调度配置的规范。
- `alert_config`：警报配置的规范。
- `objective_configs`：目标配置的规范。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn",
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
)

print(monitoring_job.gca_resource)

#### 监控作业的邮件通知。

一个邮件通知会发送到警报配置中的电子邮件地址，通知模型监控作业现在已启用。

邮件内容将会是：

<blockquote>
您好，Vertex AI 客户，

您收到这封邮件是因为您正在使用 Vertex AI 模型监控服务。
这封邮件是告知您我们已收到您设置漂移或倾斜检测的请求，用于下面列出的预测端点。从现在开始，传入的预测请求将被抽样并记录用于分析。
原始请求和响应将从预测服务中收集并保存在 bq://[your-project-id].model_deployment_monitoring_[endpoint-id].serving_predict。
</blockquote>

#### 监控任务状态

在开始`Vertex AI Model Monitoring`作业之后，直到计算出`input schema`和`skew distribution baselines`之前，作业将处于`PENDING`状态。这个过程是依次进行的。在这个示例中，使用自动生成的`input schema`时，服务会保持在`PENDING`状态，直到发送1000个预测请求（后文会讨论）为止。

一旦生成了`input schema`，则会启动一个批处理作业来从训练数据生成分布基线。同样，服务会在计算基线分布之前保持在`PENDING`状态。

一旦基线分布生成，监控作业将进入`OFFLINE`状态。按照间隔的基础 - 例如，每小时一次，监控作业会在分析采样数据时进入`RUNNING`状态。完成后，它将返回到`OFFLINE`状态，等待下一次计划的分析。

In [ ]:
jobs = monitoring_job.list(filter="display_name=churn")
job = jobs[0]
print(job.state)

## 初始化解析以自动生成输入架构

当您的“端点”接收到1000个预测请求后，建模服务将自动解析并创建“输入架构”。

### 创建1000个实例数据

在本示例中，BigQuery训练数据中的前1000条记录被用作前1000个预测请求。

*注意:* 在这种情况下，每个实例是一个预测请求。换句话说，发送1000个单个实例的预测请求与发送包含1000个实例的单个预测请求是相同的。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        instance[key] = value
    instances.append(instance)

print(len(instances))

### 发起初始预测请求

接下来，您可以使用`predict()`方法向您的`Vertex AI Endpoint`资源发送1000个预测请求。

In [ ]:
response = endpoint.predict(instances=instances)

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 输入模式的自动生成

在模型监控服务收到1000个预测请求实例后，监控将开始分析预测请求，自动生成特征输入的`输入模式`。

### 基线分布的自动生成

在`输入模式`生成后，监控服务将创建一个批处理作业来分析训练数据，以确定基线分布。

In [ ]:
# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    import time

    time.sleep(120)

监控的BigQuery表位置

用于记录采样请求的BigQuery表位于：

`<PROJECT_ID>.model_deployment_monitoring_<ENDPOINT_ID>`.serving_predict,

其中<ENDPOINT_ID>是`Vertex AI端点`资源的数字标识符。

In [ ]:
ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

BQ_MON_TABLE = f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"

table = bigquery.TableReference.from_string(BQ_MON_TABLE)
bq_table = bqclient.get_table(table)

print(bq_table)

暂停监视任务

您可以使用`pause()`和`resume()`方法分别暂停和恢复监视任务。

In [ ]:
monitoring_job.pause()
monitoring_job.resume()

### 列出监控任务

接下来，您可以使用 `list()` 方法获取所有监控任务的列表。

In [ ]:
monitoring_jobs = aiplatform.ModelDeploymentMonitoringJob.list()
print(monitoring_jobs)

### 通过筛选器列出监控任务

或者，您可以使用`filter`参数来列出一部分任务。在本例中，您可以通过监控任务的显示名称来筛选列表。

In [ ]:
monitoring_jobs = aiplatform.ModelDeploymentMonitoringJob.list(
    filter="display_name=churn"
)

print(monitoring_jobs[0].gca_resource)

### 删除监控任务

您可以使用 `delete()` 方法来删除监控任务。

*注意：* 当监控任务处于“运行”状态时，您无法删除该任务。您必须先暂停该任务。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

删除已记录的采样数据。

In [ ]:
# Delete the monitoring logged data BigQuery dataset

! bq rm -r -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}

### 使用预定义输入模式创建监控作业

接下来，您将创建另一个监控作业。这次您将加载一个预定义的`输入模式`。加载后，监控服务将使用这个`输入模式`，而不是从前1000个预测实例自动生成一个。

#### 创建预定义输入模式

预定义的`输入模式`被指定为一个YAML文件。在这个例子中，您检索用于训练数据的BigQuery模式，其中包括特征名称和数据类型，以生成YAML规范。预定义的`输入模式`必须加载到Cloud Storage位置。

了解更多关于[自定义实例模式以解析输入](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview#custom-input-schemas)。

In [ ]:
# Get the BQ table

table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])
bq_table = bqclient.get_table(table)

yaml = """type: object
properties:
"""

schema = bq_table.schema
for feature in schema:
    if feature.name == TARGET:
        continue
    if feature.field_type == "STRING":
        f_type = "string"
    else:
        f_type = "integer"
    yaml += f"""  {feature.name}:
    type: {f_type}
"""

yaml += """required:
"""
for feature in schema:
    if feature.name == TARGET:
        continue
    yaml += f"""- {feature.name}
"""

print(yaml)

with open("schema.yaml", "w") as f:
    f.write(yaml)

! gsutil cp schema.yaml {BUCKET_URI}/schema.yaml

### 创建监控作业

最后，您使用 `create()` 方法创建监控作业，并添加以下附加参数：

- `analysis_instance_schema_uri`：包含 `输入模式` 的 YAML 文件的位置。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn",
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    analysis_instance_schema_uri=f"{BUCKET_URI}/schema.yaml",
)

print(monitoring_job)

### 删除监控任务

您可以使用 `delete()` 方法来删除监控任务。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

取消部署并删除`Vertex AI Endpoint`资源

您可以使用`delete()`方法删除您的`Vertex AI Endpoint`资源。在删除之前，必须先取消部署到您的`Vertex AI Endpoint`资源的任何模型。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

删除`Vertex AI模型`资源

您可以使用`delete（）`方法删除您的`Vertex AI模型`资源。

In [ ]:
model.delete()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}

! rm -f schema.yaml

! bq rm -r -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}